Use this notebook to run and analyze the results from the xbeach model.

* Try to match the wave heights first. Also need to be checking the depth averaged velocity but that isn't measured at that many locations
* Try to get a general fit with this model not an exact one


Author: WaveHello

Date: 07/15/2024

In [ ]:
# import default modules
import numpy as np
import matplotlib.pyplot as plt
import sys
import os
import pandas as pd
import math

# Set global constants
PI = np.pi

# method to import xbtools with try routine
try:
    import xbTools
except ImportError:
    print('**no xbTools installation found in environment, adding parent path of notebook to see if it works')
    sys.path.append(r"..\..\..\xbeach-toolbox")

In [ ]:
from xbTools.xbeachpost import XBeachModelAnalysis
from xbTools.general.executing_runs import generate_batch_script, run_batch_script

In [ ]:
# Functions 
def get_script_directory():
    try:
        # This will work if the script is run directly
        script_path = os.path.abspath(__file__)
    except NameError:
        # This will work in an interactive environment like Jupyter
        script_path = os.path.abspath('')
    
    return script_path

In [ ]:
# Import the classes that represent the runs
# Add the library to the path
sys.path.append(r"..\..\..\BarSed_Lib")

# Import the library modules

from lib.data_classes.Run import Run

In [ ]:
# Import the raw bathymetry
# Set the important paths
barsed_data_path = r"D:\ERDC\BarSed_Data"

# Set the information needed for specifying the run
# Run number
run_number = "001"

# Run id
run_id = f"RUN{run_number}"

# Mat file for the first run
run_name = r"{}.mat".format(run_id)

# Name of the folder containing the wave data
WG_data_folder_name = r"WG" 

# Path to where I'm storing the wave data
WG_data_path = os.path.join(barsed_data_path, WG_data_folder_name)

# Path to the selected wave gauge data
wave_data_path = os.path.join(WG_data_path, run_name)

print(f"Run{run_number} wave mat file path: {wave_data_path}")

Run_data = Run(id = run_id, wave_file_path=wave_data_path,
              ADV_file_path = None)

print(Run_data)
print()
Run_data.load_wave_data()


In [ ]:
# Construct the wse elevation as mesured by the wave gauges 
# (doesn't include wave maker)
Run_data.construct_wave_gauge_wse()

# Make an array of all the wave gauge locations
Run_data.get_wave_gauge_locations()

# Construct the elevation of the flume as measured by the wave gauges and 
# the wave maker
Run_data.construct_flume_wse()

# get the lab wave gauge locations
lab_xdir = Run

# Don't include the wave maker
# Get the lab wave gauge flume wse
lab_wse = Run_data.flume_wse[:, 1:]
lab_wg_locs = Run_data.flume_wse_locs[0, 1:]


In [ ]:
print(lab_wg_locs)

In [ ]:
# Get the script directory
# Get the folder of the current script
script_dir = get_script_directory()

# Generate the model directory
model_dir = os.path.join(script_dir, f"")

# Need to batch file path
batch_file_name = "run_model.bat"
batch_file_path = os.path.join(model_dir, batch_file_name)

# Run the batch file
run_batch_script(batch_file_path, flag_print_Blog = False)



In [ ]:
# Store the xbeach data
results = XBeachModelAnalysis(fname = "foo", 
                              model_path=model_dir)

In [ ]:
results.set_save_fig(False)
results.set_plot_localcoords(False)
results.set_plot_km_coords(False)

In [ ]:
results.load_modeloutput("u")

In [ ]:
results.var.keys()
results.load_modeloutput("zs")
results.load_modeloutput("zb")

In [ ]:
model_time = results.var["globaltime"]

left_boundary_zs =  model_zs = results.var["zs"][:, 0, 0]


In [ ]:
model_xdir = results.var["globalx"][0, :]
print(model_xdir)

In [ ]:
nrows = 5
fig, axs = plt.subplots(nrows = nrows, ncols = 1, figsize = (8,nrows * (1.6)))

# Convert the axs to an array so I can index it, helps if I add more subplots
axs = np.atleast_1d(axs)

# Calc an equal step size to fill the plots
step = math.ceil(len(model_time) / nrows)

# Get the time indices of the model_time arr to plot the data at
time_indices = [val for val in range(0, len(model_time), step)]

# Get the location of the xgrid points
model_xdir = results.var["globalx"][0, :]

# Plot each of surface vs. plots
for i, time_index in enumerate(time_indices):
    # Get the plot time
    time = model_time[time_index]

    # Set the surface elevation at the selected time
    model_zs = results.var["zs"][time_index, 0, :]

    # Plot the surface elevation w/ time
    axs[i].plot(model_xdir, model_zs, label = "Model")
    
    # Plot the measured data
    axs[i].plot()

    if True:
        axs[i].set_title(f"Time: {time}")
    if False:
        # Format the plot
        axs[i].set_xlabel("Time (s)")
        axs[i].set_ylabel("Surface elevation (m)")

plt.tight_layout()
plt.show()
# Compare it against the lab data

In [ ]:
# Load the wave maker data
df_wave_maker = pd.read_csv("wave_maker.csv")
display(df_wave_maker)

# Get the time of lab measurements
lab_time = df_wave_maker["time"]

In [ ]:
# Plot the data at the prescribed boundary
fig, axs = plt.subplots(nrows = 1, ncols = 1, figsize = (8, 4))

axs = np.atleast_1d(axs)


i = 0
axs[i].plot(lab_time, df_wave_maker["wse"], 
            linestyle = "dashed", label = "Measured")
axs[i].plot(model_time, left_boundary_zs, label = "Model")

# Format the plot
axs[i].legend()
axs[i].set_title("Surface Elev. at left boundary")
axs[i].set_xlabel("Time (s)")
axs[i].set_ylabel("Surface Elevation")

plt.tight_layout()
plt.show()

In [ ]:
lab_time

In [ ]:
def find_closest(array, target):
    # Convert array to a NumPy array if it is not already
    array = np.array(array)
    
    # Compute the absolute differences between each element and the target
    diff = np.abs(array - target)
    
    # Find the index of the minimum difference
    idx = diff.argmin()
    
    # Get the value at that index
    closest_value = array[idx]
    
    return closest_value, idx

In [ ]:
zs0 = results.var["zs"][0, 0, 0]
zb  = results.var["zb"]

# get the xgrid
x_grd = results.var["globalx"][0]
print(x_grd)

In [ ]:
zb[0, 0, :]

In [ ]:
from scipy.interpolate import interp1d

# Make a function for the bathymatery
# bathy_func now takes in an x - location and outputs the corresponding z location
bathy_func = interp1d(x_grd, zb[0, 0, :])



In [ ]:
nrows = 3
fig_height_scale = 3 #1.6
fig, axs = plt.subplots(nrows = nrows, ncols = 1, 
                        figsize = (8,nrows * fig_height_scale),
                        dpi = 300)

# Convert the axs to an array so I can index it, helps if I add more subplots
axs = np.atleast_1d(axs)

# Calc an equal step size to fill the plots
step = math.ceil(len(model_time) / nrows)

# Get the time indices of the model_time arr to plot the data at
time_indices = [val for val in range(0, len(model_time), step)]

# Get the location of the xgrid points
model_xdir = results.var["globalx"][0, :]

# Plot each of surface vs. plots
for i, time_index in enumerate(time_indices):
    # Get the plot time
    model_t = model_time[time_index]

    # Get the closest lab time
    lab_t, lab_index = find_closest(lab_time, model_t)

    # Set the surface elevation at the selected time
    model_zs = results.var["zs"][time_index, 0, :]

    # Plot the surface elevation w/ time
    axs[i].plot(model_xdir, model_zs, label = "Model")

    # Make a copy of the wse elevation from the gauges
    lab_wse_copy = lab_wse.copy()

    # move the wave gauges up by the wse - except for the last one, it's too far too the right
    # for it to measure the d0 water level
    lab_wse_copy[lab_index, :-1] = lab_wse_copy[lab_index, :-1] + zs0

    lab_wse_copy[lab_index, -1] = lab_wse_copy[lab_index, -1] + bathy_func(lab_wg_locs[-1])

    # Plot the measured data
    axs[i].scatter(lab_wg_locs, lab_wse_copy[lab_index, :], 
                label = "lab data {:.2f}".format(lab_t),
                marker = ".")
    
    # Plot the bathymtry
    
    # Selecting if only the waves should be plotted
    if True:
        axs[i].plot(model_xdir, zb[time_index, 0, :], label = "bathy", color ="grey")
    else:
        # Select only the waves
        axs[i].set_ylim(zs0-0.8, 4)

    # Setting the legend
    if True:
        axs[i].legend()

    if True:
        axs[i].set_title("Time: {:.2f}".format(model_t))

    # Setting x and y labels - get's kind of messy when there are lots of plots
    if True:
        # Format the plot
        axs[i].set_xlabel("x-direction (m)", fontweight = "bold")
        axs[i].set_ylabel("Elevation (m)", fontweight = "bold")

plt.suptitle("Comparison of xBeach Non-Hydro. vs. BarSed lab data \n(Hydrodynamics Only)", fontweight = "bold")
plt.tight_layout()
plt.show()
# Compare it against the lab data

In [ ]:
bathy_func(lab_wg_locs[-1])

In [ ]:
results.load_modeloutput("u")
u = results.var["u"]

In [ ]:
df = pd.read_csv("boun_U.bcf", sep = "\\s+", skiprows=3, header=None, names=["t", "zs", "u"])

In [ ]:
plt.plot(model_time, u[:, 0, 0], label = "xBeach")
plt.plot(df["t"], df["u"], label = "input")
plt.title("Left boundary input vs. xbeach output")
plt.legend()
plt.show()

In [ ]:
print(Run_data.wave_gauges[-1])